In [20]:
import torch
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os

In [21]:
base_path = '..'
# Gets label
csv_path = os.path.join(base_path,'data/3DYoga90_corrected.csv')

SAVE_PATH = os.path.join(base_path, 'biomechanical_features')
os.makedirs(SAVE_PATH, exist_ok=True)

# Classification classes
pose_list = ['mountain', 'half-way-lift', 'standing-forward-bend', 'downward-dog']
subset_of_poses = pose_list
NUM_CLASSES = len(pose_list)

dataset_dir = os.path.join(base_path, 'official_dataset')
assert os.path.isdir(dataset_dir), f"Directory '{dataset_dir}' does not exist."

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [22]:
meta_info_path = os.path.join(base_path, 'data')
pose_index = pd.read_csv(f'{meta_info_path}/pose-index.csv')
sequence_index = pd.read_csv(f'{meta_info_path}/3DYoga90_corrected.csv')

In [23]:
# Keep only relevant columns
def read_meta_data():
    meta_info_path = os.path.join(base_path, 'data')
    pose_index = pd.read_csv(f'{meta_info_path}/pose-index.csv')
    sequence_index = pd.read_csv(f'{meta_info_path}/3DYoga90_corrected.csv')
    parquet_index = sequence_index[['sequence_id', 'l3_pose', 'split']]
    return parquet_index

In [24]:

import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class Yoga3DDataset(Dataset):
    def __init__(self, parquet_index, root_dir =  dataset_dir,subset_of_poses= subset_of_poses, sub_sampling_length = 20, transform=None, max_frames=None):
        self.parquet_index = parquet_index
        self.parquet_index = self.parquet_index[self.parquet_index['l3_pose'].isin(subset_of_poses)]
        self.root_dir = root_dir
        self.transform = transform
        self.max_frames = max_frames
        self.sub_sampling_length = sub_sampling_length
        self.pose_to_label = {pose: i for i, pose in enumerate(subset_of_poses)}
        self.use_augmentation = False

        self.cache = dict()
        self.idx_to_seq = dict()

    def __len__(self):
        return len(self.parquet_index)

    def __getitem__(self, idx):
        if idx in self.cache:
            data, label = self.cache[idx]
        else:
            fname, pose_name, _ = self.parquet_index.iloc[idx]
            label = self.pose_to_label[pose_name]
            path = os.path.join(self.root_dir, f'{fname}.parquet')

            df = pd.read_parquet(path)
            df = df.drop(columns=['frame', 'row_id', 'type','landmark_index'])

            data = self.to_tensor(df)
            # data = self.sub_sample(data)
            data = data.permute(1,0,2)
            self.cache[idx] = (data, label)
            self.idx_to_seq[idx] = fname

        if self.transform and self.use_augmentation:
            data = self.transform(data.clone())

        return data, self.idx_to_seq[idx] # C, T , V

    def sub_sample(self, data):
        # data(Number_of_frames, 3, 33)
        total_frames = data.shape[0]
        indices = torch.linspace(0, total_frames -1 , self.sub_sampling_length, dtype= int)
        return data[indices]

    def to_tensor(self, df):
        # Reshape the data to (num_frames, num_landmarks, 3)  ## WHAT WHAT? this doesn't make sense remove this line you are doing (number of frames, 3 , 33)
        num_frames = len(df) // 33  # Assuming 33 landmarks per frame
        data = df.values.reshape(num_frames, 33, 3)
        return torch.FloatTensor(data).permute(0, 2, 1)

In [25]:
import torch
import numpy as np
from typing import Dict, List, Tuple

class BiomechanicalFeatureExtractor:
    def __init__(self):
        """
        Initialize the feature extractor.
        Note: Simplified version using frame-by-frame differences
        """
        pass
    
    def compute_velocities(self, joint_positions: torch.Tensor) -> torch.Tensor:
        """
        Compute joint velocities from positions using simple frame differences.
        
        Args:
            joint_positions: Tensor of shape (frames, num_joints, 3)
            
        Returns:
            Tensor of shape (frames, num_joints, 3) containing velocities
        """
        # Initialize velocities tensor with zeros
        velocities = torch.zeros_like(joint_positions)
        
        # Compute velocities as simple differences between consecutive frames
        velocities[:-1] = joint_positions[1:] - joint_positions[:-1]
        
        # For the last frame, use the same velocity as the second-to-last frame
        velocities[-1] = velocities[-2]
        
        return velocities
    
    def compute_accelerations(self, velocities: torch.Tensor) -> torch.Tensor:
        """
        Compute joint accelerations from velocities using simple frame differences.
        
        Args:
            velocities: Tensor of shape (frames, num_joints, 3)
            
        Returns:
            Tensor of shape (frames, num_joints, 3) containing accelerations
        """
        # Initialize accelerations tensor with zeros
        accelerations = torch.zeros_like(velocities)
        
        # Compute accelerations as simple differences between consecutive velocities
        accelerations[:-1] = velocities[1:] - velocities[:-1]
        
        # For the last frame, use the same acceleration as the second-to-last frame
        accelerations[-1] = accelerations[-2]
        
        return accelerations
    
    def compute_joint_angle(self, p1: torch.Tensor, p2: torch.Tensor, p3: torch.Tensor) -> torch.Tensor:
        """
        Compute the angle between three joints.
        
        Args:
            p1, p2, p3: Joint positions forming the angle (p2 is the vertex)
            
        Returns:
            Angle in radians
        """
        # Compute vectors
        v1 = p1 - p2  # Vector from p2 to p1
        v2 = p3 - p2  # Vector from p2 to p3
        
        # Compute dot product
        dot_product = torch.sum(v1 * v2, dim=-1)
        
        # Compute magnitudes
        v1_mag = torch.sqrt(torch.sum(v1 * v1, dim=-1))
        v2_mag = torch.sqrt(torch.sum(v2 * v2, dim=-1))
        
        # Compute cosine of angle
        cos_angle = dot_product / (v1_mag * v2_mag)
        
        # Clamp to avoid numerical issues
        cos_angle = torch.clamp(cos_angle, -1.0, 1.0)
        
        # Return angle in radians
        return torch.acos(cos_angle)
    
    def compute_joint_angles(
        self, 
        joint_positions: torch.Tensor,
        joint_triplets: List[Tuple[int, int, int]]
    ) -> torch.Tensor:
        """
        Compute angles for all specified joint triplets.
        
        Args:
            joint_positions: Tensor of shape (frames, num_joints, 3)
            joint_triplets: List of tuples containing indices of joint triplets
            
        Returns:
            Tensor of shape (frames, num_angles, 1) containing angles
        """
        num_frames = joint_positions.shape[0]
        num_angles = len(joint_triplets)
        angles = torch.zeros((num_frames, num_angles, 1))
        
        for i, (j1, j2, j3) in enumerate(joint_triplets):
            angle = self.compute_joint_angle(
                joint_positions[:, j1],
                joint_positions[:, j2],
                joint_positions[:, j3]
            )
            angles[:, i, 0] = angle
            
        return angles
    
    def extract_features(
        self, 
        joint_positions: torch.Tensor,
        joint_triplets: List[Tuple[int, int, int]]
    ) -> Dict[str, torch.Tensor]:
        """
        Extract all biomechanical features from joint positions.
        
        Args:
            joint_positions: Tensor of shape (frames, num_joints, 3)
            joint_triplets: List of tuples containing indices of joint triplets
            
        Returns:
            Dictionary containing all computed features
        """
        # Compute velocities
        velocities = self.compute_velocities(joint_positions)
        
        # Compute accelerations
        accelerations = self.compute_accelerations(velocities)
        
        # Compute joint angles
        # angles = self.compute_joint_angles(joint_positions, joint_triplets)
        
        # Return features dictionary
        return {
            "Joint Position": joint_positions,
            # "Joint Angles": angles,
            "Joint Velocity": velocities,
            "Joint Acceleration": accelerations
        }
    
    @staticmethod
    def save_features(features: Dict[str, torch.Tensor], filename: str):
        """Save features to a .pt file."""
        torch.save(features, filename)
        print(f"saving {filename}")
    
    @staticmethod
    def load_features(filename: str) -> Dict[str, torch.Tensor]:
        """Load features from a .pt file."""
        return torch.load(f"{filename}.pt")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import os
from tqdm import tqdm
from typing import Dict, List, Tuple

def process_dataset(
    dataset: Yoga3DDataset,
    output_dir: str,
    batch_size: int = 1,
    num_workers: int = 1,
    joint_triplets: List[Tuple[int, int, int]] = None
) -> None:
    """
    Process entire dataset and save biomechanical features.
    
    Args:
        dataset: Yoga3DDataset instance
        output_dir: Directory to save extracted features
        batch_size: Batch size for dataloader (default 1 for sequential processing)
        num_workers: Number of worker processes for data loading
        joint_triplets: List of joint triplet indices for angle calculation
    """
    
    extractor = BiomechanicalFeatureExtractor()
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False  # Keep sequential order
    )
    
    for batch in tqdm(dataloader, desc="Processing sequences"):
        joints, fnames = batch  # Shape: (B, 3, frames, 33) 
        for i in range(len(joints)):
            joint_positions = joints[i]  # Shape: (3, frames, 33)
            fname = fnames[i]
            # From: (3, frames, 33) -> To: (frames, 33, 3)
            joint_positions = joint_positions.permute(1, 2, 0)
            # Extract features
            features = extractor.extract_features(
                joint_positions,
                joint_triplets
            )
            print(features["Joint Velocity"])  
            # Save features
            output_path = os.path.join(output_dir, f"{fname}.pt")
            extractor.save_features(features, output_path)

def verify_processed_features(
    output_dir: str,
    expected_count: int
) -> bool:
    """
    Verify that all features were properly extracted and saved.
    
    Args:
        output_dir: Directory containing saved features
        expected_count: Expected number of processed sequences
        
    Returns:
        bool: True if verification passes
    """
    # Count .pt files in output directory
    feature_files = list(Path(output_dir).glob("*.pt"))
    actual_count = len(feature_files)
    
    # Basic verification
    if actual_count != expected_count:
        print(f"Warning: Found {actual_count} feature files, expected {expected_count}")
        return False
    
    # Try loading each file
    for file_path in tqdm(feature_files, desc="Verifying files"):
        try:
            features = torch.load(str(file_path))
            # Check if all expected keys are present
            expected_keys = {"Joint Position", "Joint Angles", "Joint Velocity", "Joint Acceleration"}
            if not all(key in features for key in expected_keys):
                print(f"Warning: Missing keys in {file_path}")
                return False
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return False
    
    return True

if __name__ == "__main__":
    dataset = Yoga3DDataset(read_meta_data())
   
    process_dataset(
        dataset=dataset,
        output_dir=SAVE_PATH,
        batch_size=1,
        num_workers=1,
        joint_triplets=None  
    )
    
    # Verify processing
    # success = verify_processed_features(
    #     output_dir=SAVE_PATH,
    #     expected_count=len(dataset)
    # )
    
    # if success:
    #     print("Feature extraction completed successfully!")
    # else:
    #     print("Feature extraction completed with some issues.")

In [27]:

# import os
# import torch
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# from mpl_toolkits.mplot3d import Axes3D

# # Define the MediaPipe Pose landmarks and their connections
# # Landmark names and their corresponding indices
# LANDMARK_NAMES = [
#     'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner',
#     'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left',
#     'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
#     'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index',
#     'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip',
#     'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel',
#     'right_heel', 'left_foot_index', 'right_foot_index'
# ]

# # Define connections between landmarks
# POSE_CONNECTIONS = [
#     (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8),
#     (1, 2), (2, 3), (4, 5), (5, 6), (7, 9), (8, 10), (11, 12),
#     (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (12, 14),
#     (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24),
#     (23, 24), (23, 25), (24, 26), (25, 27), (26, 28), (27, 29),
#     (28, 30), (29, 31), (30, 32)
# ]

# def visualize_pose_sequence(data, label, connections=POSE_CONNECTIONS, figsize=(8, 8)):
#     """
#     Visualize a sequence of 3D poses.

#     Parameters:
#         data (torch.Tensor): Tensor of shape (channels, number_of_frames, landmarks=33)
#                              where channels are x, y, z coordinates.
#         label (int): The label corresponding to the pose.
#         connections (list of tuples): Landmark connections to draw the skeleton.
#         figsize (tuple): Figure size for the plot.
#     """
#     # Convert tensor to numpy array
#     data_np = data.numpy()
#     # Assuming channels are in the order x, y, z
#     x = data_np[0]  # shape: (number_of_frames, 33)
#     y = data_np[1]
#     z = data_np[2]
#     # print(x.shape)
#     num_frames = x.shape[0]

#     # Create a figure and a 3D subplot
#     fig = plt.figure(figsize=figsize)
#     ax = fig.add_subplot(111, projection='3d')
#     plt.title(f'Pose: {label}')

#     # Function to update the plot for each frame
#     def update(frame):
#         ax.cla()
#         ax.set_xlim3d(-1, 1)
#         ax.set_ylim3d(-1, 1)
#         ax.set_zlim3d(-1, 1)
#         ax.set_xlabel('X')
#         ax.set_ylabel('Y')
#         ax.set_zlabel('Z')
#         plt.title(f'Pose: {label}, Frame: {frame + 1}/{num_frames}')

#         # Scatter plot of landmarks
#         ax.scatter(x[frame], y[frame], z[frame], c='r', marker='.')

#         # Draw connections
#         for connection in connections:
#             idx1, idx2 = connection
#             ax.plot([x[frame, idx1], x[frame, idx2]],
#                     [y[frame, idx1], y[frame, idx2]],
#                     [z[frame, idx1], z[frame, idx2]], 'b-')
#         # plt.show() 
#         return

#     # Create the animation
#     anim = FuncAnimation(fig, update, frames=num_frames, interval=100)
#     plt.show()
#     save_dir = 'data'
#     os.makedirs(save_dir, exist_ok=True)
#     save_path = os.path.join(save_dir, f'test_{label}.mp4')
#     anim.save(save_path)